# Gender recognition

In [2]:
!pip install pandas
!pip install sklearn
!pip install scipy

In [2]:
!rm -rf data
!mkdir data
%cd data
!wget "http://datax.kennesaw.edu/imdb_wiki/imdb4p5.csv"
!wget "http://datax.kennesaw.edu/imdb_wiki/wiki5.csv"
%cd ..
!ls data

mkdir: cannot create directory ‘data’: File exists
/content/data
--2018-09-05 17:33:35--  http://datax.kennesaw.edu/imdb_wiki/wiki5.csv
Resolving datax.kennesaw.edu (datax.kennesaw.edu)... 168.28.189.125
Connecting to datax.kennesaw.edu (datax.kennesaw.edu)|168.28.189.125|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 112721725 (107M) [text/csv]
Saving to: ‘wiki5.csv’

wiki5.csv           100%[===================>] 107.50M  22.6MB/s    in 5.6s    

2018-09-05 17:33:41 (19.2 MB/s) - ‘wiki5.csv’ saved [112721725/112721725]

/content
imdb4p5.csv  wiki5.csv


In [0]:
WIKI_SET=data/wiki5.csv
IMDB_SET=data/imdb4p5.csv

In [0]:
import pandas as pd
import numpy as np


class DataLoader:
    """
    Loads and parses data
    """
    def __init__(self):
        self.data = None
        self.X = None
        self.y = None
        self.read_csv("data/wiki5.csv")

    def read_csv(self, filepath):
        self.data = pd.read_csv(filepath)

    def get_data(self):
        self.y = self.data['gender'].values
        self.X = self.data.drop(['gender', 'age'], axis=1).values
        self.X = np.delete(self.X, np.s_[0], axis=1)
        return self.X, self.y


In [7]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

print("Loading data")
dl = DataLoader()
X, y = dl.get_data()

print("Creating the mlp classifier")
mlp = MLPClassifier(hidden_layer_sizes=(2048, 1024, 512, 254, 128, 64, 32, 16, 8, 4), verbose=True)
print("Created classifier")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
print("Training started")
mlp.fit(X_train, y_train)
print("Training done")
print(mlp.score(X_test, y_test))



Loading data
Creating the mlp classifier
Created classifier
Training started
Iteration 1, loss = 3.03562594
Iteration 2, loss = 1.67831565
Iteration 3, loss = 0.69429264
Iteration 4, loss = 0.69403117
Iteration 5, loss = 0.69376210
Iteration 6, loss = 0.69346794
Iteration 7, loss = 0.69315282
Iteration 8, loss = 0.69285618
Iteration 9, loss = 0.69256268
Iteration 10, loss = 0.69223848
Iteration 11, loss = 0.69192835
Iteration 12, loss = 0.69163885
Iteration 13, loss = 0.69132520
Iteration 14, loss = 0.69104932
Iteration 15, loss = 0.69076185
Iteration 16, loss = 0.69047351
Iteration 17, loss = 0.69020865
Iteration 18, loss = 0.68993679
Iteration 19, loss = 0.68967929
Iteration 20, loss = 0.68944089
Iteration 21, loss = 0.68921739
Iteration 22, loss = 0.68896706
Iteration 23, loss = 0.68875964
Iteration 24, loss = 0.68853999
Iteration 25, loss = 0.68832264
Iteration 26, loss = 0.68813769
Iteration 27, loss = 0.68795173
Iteration 28, loss = 0.68777841
Iteration 29, loss = 0.68760590
Iter

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from sklearn.model_selection import train_test_split
import keras.losses
import numpy as np
from keras.utils import to_categorical

print("Loading data")
dl = DataLoader()
X, y = dl.get_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train = np.asarray([sample.reshape(100, 100) for sample in X_train])
X_test = np.asarray([sample.reshape(100, 100) for sample in X_test])
X_train = X_train.reshape(X_train.shape[0], 100, 100, 1)
X_test = X_test.reshape(X_test.shape[0], 100, 100, 1)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


print(X_train.shape)
print(X_test.shape)

input_shape = (100, 100, 1)


num_classes = 2

model = Sequential()

model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.01),
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=32,
          epochs=100,
          verbose=1)

score = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Using TensorFlow backend.


Loading data
(2150, 100, 100, 1)
(1059, 100, 100, 1)
Epoch 1/100
2150/2150 [==============================] - 5s 2ms/step - loss: 6.8837 - acc: 0.5693
Epoch 2/100
2150/2150 [==============================] - 3s 1ms/step - loss: 6.9420 - acc: 0.5693
Epoch 3/100
2150/2150 [==============================] - 3s 1ms/step - loss: 6.9420 - acc: 0.5693
Epoch 4/100
2150/2150 [==============================] - 3s 1ms/step - loss: 6.9420 - acc: 0.5693
Epoch 5/100
2150/2150 [==============================] - 3s 1ms/step - loss: 6.9420 - acc: 0.5693
Epoch 6/100
2150/2150 [==============================] - 3s 1ms/step - loss: 6.9420 - acc: 0.5693
Epoch 7/100
2150/2150 [==============================] - 3s 1ms/step - loss: 6.9420 - acc: 0.5693
Epoch 8/100
2150/2150 [==============================] - 3s 1ms/step - loss: 6.9420 - acc: 0.5693
Epoch 9/100
2150/2150 [==============================] - 3s 1ms/step - loss: 6.9420 - acc: 0.5693
Epoch 10/100
2150/2150 [==============================] - 3s 1ms/